<a href="https://colab.research.google.com/github/na2003-gif/The-impact-of-Environmental-Protection-Expenditures-on-Environmental-Improvement/blob/main/Landmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install linearmodels i

In [63]:
import numpy as np
import pandas as pd

from scipy.stats import normaltest, jarque_bera
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [64]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from linearmodels.panel import PanelOLS

In [76]:
data =pd.read_excel("/content/Land_final.xlsx")
df = pd.DataFrame(data)

# =====================================================
# DATA CLEANING
# =====================================================
def winsorize_series(s: pd.Series, p: float = 0.05) -> pd.Series:
    lo, hi = s.quantile(p), s.quantile(1 - p)
    return s.clip(lo, hi)

def log1p_nonneg(s: pd.Series) -> pd.Series:
    return np.log1p(s.clip(lower=0))


quant_cols = ["total","agr_land","forestry_land","special_land","resi_land","others_land","hdi","density","epe","ipi"]
for c in quant_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["resi_share"] =  df["resi_land"] / df["total"]
df["agr_share"] =  df["agr_land"] / df["total"]
df["forestry_share"] =  df["forestry_land"] / df["total"]
df["special_share"]=df["special_land"] / df["total"]
df["others_share"] =  df["others_land"] / df["total"]
# log1p
log_cols = ["hdi","density","epe","ipi"]
for c in log_cols:
    df[f"l{c.lower()}"] = log1p_nonneg(df[c])

# Winsorization 5% (two-sided)
for c in log_cols:
    df[c] = winsorize_series(df[c], p=0.10)
    df[f"l{c.lower()}"] = winsorize_series(df[f"l{c.lower()}"], p=0.10)

print("=== After Step 1 (clean + log + winsor) ===")
print(df.head())

=== After Step 1 (clean + log + winsor) ===
          province  year         province_year   total  agr_land  \
0  Ba Ria Vung Tau  2019  Ba Ria Vung Tau 2019  198.10    104.90   
1  Ba Ria Vung Tau  2020  Ba Ria Vung Tau 2020  198.26    103.09   
2  Ba Ria Vung Tau  2021  Ba Ria Vung Tau 2021  198.26    103.09   
3  Ba Ria Vung Tau  2022  Ba Ria Vung Tau 2022  198.26    102.61   
4  Ba Ria Vung Tau  2023  Ba Ria Vung Tau 2023  198.26    102.66   

   forestry_land  special_land  resi_land  others_land   hdi  ...      ipi  \
0          33.90         33.20       7.30        18.80  0.78  ...  102.070   
1          32.17         37.36       7.78        17.86  0.78  ...   96.878   
2          32.15         37.25       7.90        17.87  0.78  ...   96.878   
3          34.26         36.92       7.98        16.49  0.78  ...  105.650   
4          34.26         36.83       8.07        16.44  0.78  ...   99.430   

   resi_share  agr_share  forestry_share  special_share  others_share  \
0    

In [66]:
drop_cols = ["total","agr_land","forestry_land","special_land","resi_land","others_land","hdi",	"density",	"epe",	"ipi"]
df.drop(columns=drop_cols, inplace=True)
df.dropna(inplace=True)


In [67]:
print("=== After Step 2 (drop na) ===")
print(df.head())

=== After Step 2 (drop na) ===
          province  year         province_year  resi_share  agr_share  \
0  Ba Ria Vung Tau  2019  Ba Ria Vung Tau 2019    0.036850   0.529531   
1  Ba Ria Vung Tau  2020  Ba Ria Vung Tau 2020    0.039241   0.519974   
2  Ba Ria Vung Tau  2021  Ba Ria Vung Tau 2021    0.039847   0.519974   
3  Ba Ria Vung Tau  2022  Ba Ria Vung Tau 2022    0.040250   0.517553   
4  Ba Ria Vung Tau  2023  Ba Ria Vung Tau 2023    0.040704   0.517805   

   forestry_share  others_share      lhdi  ldensity      lepe      lipi  
0        0.171126      0.094902  0.576613  6.368187  6.571894  4.635408  
1        0.162262      0.090084  0.576613  6.380123  6.571894  4.583721  
2        0.162161      0.090134  0.576613  6.386879  6.536634  4.583721  
3        0.172803      0.083174  0.576613  6.390241  6.375774  4.669552  
4        0.172803      0.082921  0.576613  6.396930  6.303754  4.609461  


In [68]:
df.describe()

,year,resi_share,agr_share,forestry_share,others_share,lhdi,ldensity,lepe,lipi
count,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000
mean,2020.945946,0.033797,0.377285,0.395658,0.106179,0.534398,5.643690,5.252746,4.689970
std,1.391061,0.031449,0.195057,0.259285,0.080764,0.023977,0.875473,0.741101,0.060845
min,2019.000000,0.003308,0.051362,0.000000,-0.295480,0.500775,4.522870,4.239572,4.583721
25%,2020.000000,0.010984,0.214395,0.121532,0.047121,0.518794,4.901557,4.745323,4.652483
50%,2021.000000,0.020780,0.347139,0.456442,0.082357,0.530628,5.488938,5.099866,4.693639
75%,2022.000000,0.050744,0.517679,0.597795,0.148070,0.553885,6.383501,5.673280,4.735496
max,2023.000000,0.134314,1.036406,0.850864,0.400375,0.576613,7.036481,6.571894,4.784437


**TEST FOR RESIDENTAL SHARE VALUE**

In [69]:
# =========================
# STEP 2 — DIAGNOSTICS (share_urban)
# =========================
Y = "resi_share"
X = ["lhdi", "ldensity", "lepe", "lipi"]

df_m = df[["province", "year", Y] + X].dropna().copy()
df_m["year"] = pd.to_numeric(df_m["year"], errors="coerce").astype(int)

panel = df_m.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]

# -------------------------
# Multicollinearity (VIF>10)
# -------------------------
def vif_report(X_df: pd.DataFrame):
    Xv = X_df.copy()
    Xv = Xv.dropna()
    Xv = sm.add_constant(Xv)
    out = []
    for i, col in enumerate(Xv.columns):
        out.append([col, variance_inflation_factor(Xv.values, i)])
    return pd.DataFrame(out, columns=["var","VIF"]).sort_values("VIF", ascending=False)

vif_df = vif_report(X_df)
print("\n=== Step 2.2 VIF ===")
print(vif_df)

VIF_THRESHOLD = 10
need_pca = vif_df.query("var != 'const'")["VIF"].max() > VIF_THRESHOLD

if need_pca:
    print("\n>>> VIF > 10 detected -> Step 2.2b PCA grouping")
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X_df.values)

    pca = PCA(n_components=min(2, X_df.shape[1]))
    pcs = pca.fit_transform(X_std)

    print("Explained variance ratio:", pca.explained_variance_ratio_)


    X_pca = pd.DataFrame(pcs, index=X_df.index, columns=["pc1","pc2"][:pcs.shape[1]])
    panel_pca = panel.copy()
    for c in X_pca.columns:
        panel_pca[c] = X_pca[c]
else:
    panel_pca = None

# -------------------------
# Residual diagnostics
# -------------------------

fe_2way = PanelOLS(y, X_df, entity_effects=True, time_effects=True, drop_absorbed=True)\
            .fit(cov_type="clustered", cluster_entity=True)

resid = fe_2way.resids.dropna()

print("\n=== Step 2.3 Residual diagnostics ===")
# Normality test on residuals
if len(resid) >= 8:
    nt_stat, nt_p = normaltest(resid.values)
    jb_stat, jb_p = jarque_bera(resid.values)
    print(f"Residual normaltest p = {nt_p:.4f} | JB p = {jb_p:.4f}")
else:
    print("Residual sample too small for normality tests.")

# -------------------------
# 2.4 Serial correlation test ( simple AR(1) on residuals)
# -------------------------

resid_df = resid.to_frame("resid").reset_index()  # columns: province, year, resid
resid_df["resid_lag1"] = resid_df.groupby("province")["resid"].shift(1)
ar_df = resid_df.dropna(subset=["resid","resid_lag1"]).copy()

if len(ar_df) >= 10:
    X_ar = sm.add_constant(ar_df["resid_lag1"])
    ar_model = sm.OLS(ar_df["resid"], X_ar).fit()
    p_serial = ar_model.pvalues.get("resid_lag1", np.nan)
    print("\n=== Step 2.4 Serial correlation (AR(1) on residuals) ===")
    print(ar_model.summary().tables[1])
    print(f"Serial correlation p-value (lag residual) = {p_serial:.6f}")
else:
    p_serial = np.nan
    print("\n=== Step 2.4 Serial correlation ===")
    print("Not enough data to test serial correlation.")

# -------------------------
# 2.5 Heteroskedasticity test (Breusch–Pagan on FE residuals vs regressors)
# -------------------------

if len(resid) == len(X_df):
    exog_bp = sm.add_constant(X_df)
    bp_stat, bp_p, f_stat, f_p = het_breuschpagan(resid.values, exog_bp.values)
    print("\n=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===")
    print({"LM_pvalue": bp_p, "F_pvalue": f_p})
else:
    print("\n=== Step 2.5 Heteroskedasticity ===")
    print("Residual/exog length mismatch (due to dropped absorbed vars). Skip BP or recompute on aligned index.")

# -------------------------
# 2.6 Decision
# -------------------------
print("\n=== Step 2.6 Decision ===")
if need_pca:
    print("- Multicollinearity: YES (VIF>10) -> consider using PCA components (pc1/pc2) for estimation.")
else:
    print("- Multicollinearity: NO (VIF<=10) -> keep original X.")

if not np.isnan(p_serial) and p_serial < 0.05:
    print("- Serial correlation: YES (p<0.05) -> use Cluster SE by province (recommended).")
else:
    print("- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.")

print("- Next step: Main Estimation (Two-way FE) using clustered SE.")


=== Step 2.2 VIF ===
        var          VIF
0     const  7400.940745
1      lhdi     2.649627
2  ldensity     1.948292
3      lepe     1.874003
4      lipi     1.033488

=== Step 2.3 Residual diagnostics ===
Residual normaltest p = 0.0000 | JB p = 0.0000

=== Step 2.4 Serial correlation (AR(1) on residuals) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.681e-06   3.48e-05      0.278      0.781   -5.89e-05    7.83e-05
resid_lag1    -0.0051      0.040     -0.127      0.899      -0.084       0.074
Serial correlation p-value (lag residual) = 0.898763

=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===
{'LM_pvalue': np.float64(0.001625707667532991), 'F_pvalue': np.float64(0.0014002645988316735)}

=== Step 2.6 Decision ===
- Multicollinearity: NO (VIF<=10) -> keep original X.
- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.
- N

In [70]:

# =========================
# STEP 3 — MAIN ESTIMATION
# Two-way FE + Clustered SE (province)
# =========================
Y = "resi_share"                      # bạn đổi thành "share_urban" nếu muốn
X = ["lhdi", "ldensity", "lepe", "lipi"]

df_est = df[["province", "year", Y] + X].dropna().copy()
df_est["year"] = pd.to_numeric(df_est["year"], errors="coerce").astype(int)

panel = df_est.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]

fe_2way = PanelOLS(
    y, X_df,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
).fit(
    cov_type="clustered",
    cluster_entity=True
)

print("=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===")
print(fe_2way.summary)


result_table = pd.DataFrame({
    "coef": fe_2way.params,
    "std_err": fe_2way.std_errors,
    "t": fe_2way.tstats,
    "p_value": fe_2way.pvalues
}).sort_values("p_value")

print("\n=== Coefficient table (sorted by p-value) ===")
print(result_table)


=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===
                          PanelOLS Estimation Summary                           
Dep. Variable:             resi_share   R-squared:                        0.0196
Estimator:                   PanelOLS   R-squared (Between):             -0.9811
No. Observations:                 259   R-squared (Within):              -0.0399
Date:                Mon, Dec 22 2025   R-squared (Overall):             -1.0075
Time:                        09:53:37   Log-likelihood                    1472.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.9732
Entities:                          56   P-value                           0.4233
Avg Obs:                       4.6250   Distribution:                   F(4,195)
Min Obs:                       1.0000                                           
Max Obs:                   

**TEST FOR AGRICULTURE RENT SHARE**

In [71]:
# =========================
# STEP 2 — DIAGNOSTICS (share_urban)
# =========================
Y = "agr_share"
X = ["lhdi", "ldensity", "lepe", "lipi"]

# 2.0: prepare panel data
df_m = df[["province", "year", Y] + X].dropna().copy()
df_m["year"] = pd.to_numeric(df_m["year"], errors="coerce").astype(int)

panel = df_m.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]


# -------------------------
# Multicollinearity (VIF>10)
# -------------------------
def vif_report(X_df: pd.DataFrame):
    Xv = X_df.copy()
    Xv = Xv.dropna()
    Xv = sm.add_constant(Xv)
    out = []
    for i, col in enumerate(Xv.columns):
        out.append([col, variance_inflation_factor(Xv.values, i)])
    return pd.DataFrame(out, columns=["var","VIF"]).sort_values("VIF", ascending=False)

vif_df = vif_report(X_df)
print("\n=== Step 2.2 VIF ===")
print(vif_df)

VIF_THRESHOLD = 10
need_pca = vif_df.query("var != 'const'")["VIF"].max() > VIF_THRESHOLD

if need_pca:
    print("\n>>> VIF > 10 detected -> Step 2.2b PCA grouping")
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X_df.values)

    pca = PCA(n_components=min(2, X_df.shape[1]))
    pcs = pca.fit_transform(X_std)

    print("Explained variance ratio:", pca.explained_variance_ratio_)


    X_pca = pd.DataFrame(pcs, index=X_df.index, columns=["pc1","pc2"][:pcs.shape[1]])
    panel_pca = panel.copy()
    for c in X_pca.columns:
        panel_pca[c] = X_pca[c]
else:
    panel_pca = None


# -------------------------
# Serial correlation test ( simple AR(1) on residuals)
# -------------------------

resid_df = resid.to_frame("resid").reset_index()  # columns: province, year, resid
resid_df["resid_lag1"] = resid_df.groupby("province")["resid"].shift(1)
ar_df = resid_df.dropna(subset=["resid","resid_lag1"]).copy()

if len(ar_df) >= 10:
    X_ar = sm.add_constant(ar_df["resid_lag1"])
    ar_model = sm.OLS(ar_df["resid"], X_ar).fit()
    p_serial = ar_model.pvalues.get("resid_lag1", np.nan)
    print("\n=== Step 2.4 Serial correlation (AR(1) on residuals) ===")
    print(ar_model.summary().tables[1])
    print(f"Serial correlation p-value (lag residual) = {p_serial:.6f}")
else:
    p_serial = np.nan
    print("\n=== Step 2.4 Serial correlation ===")
    print("Not enough data to test serial correlation.")

# -------------------------
# Heteroskedasticity test (Breusch–Pagan on FE residuals vs regressors)
# -------------------------

if len(resid) == len(X_df):
    exog_bp = sm.add_constant(X_df)
    bp_stat, bp_p, f_stat, f_p = het_breuschpagan(resid.values, exog_bp.values)
    print("\n=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===")
    print({"LM_pvalue": bp_p, "F_pvalue": f_p})
else:
    print("\n=== Step 2.5 Heteroskedasticity ===")
    print("Residual/exog length mismatch (due to dropped absorbed vars). Skip BP or recompute on aligned index.")

# -------------------------
# Decision
# -------------------------
print("\n=== Step 2.6 Decision ===")
if need_pca:
    print("- Multicollinearity: YES (VIF>10) -> consider using PCA components (pc1/pc2) for estimation.")
else:
    print("- Multicollinearity: NO (VIF<=10) -> keep original X.")

if not np.isnan(p_serial) and p_serial < 0.05:
    print("- Serial correlation: YES (p<0.05) -> use Cluster SE by province (recommended).")
else:
    print("- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.")

print("- Next step: Main Estimation (Two-way FE) using clustered SE.")


=== Step 2.2 VIF ===
        var          VIF
0     const  7400.940745
1      lhdi     2.649627
2  ldensity     1.948292
3      lepe     1.874003
4      lipi     1.033488

=== Step 2.4 Serial correlation (AR(1) on residuals) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.681e-06   3.48e-05      0.278      0.781   -5.89e-05    7.83e-05
resid_lag1    -0.0051      0.040     -0.127      0.899      -0.084       0.074
Serial correlation p-value (lag residual) = 0.898763

=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===
{'LM_pvalue': np.float64(0.001625707667532991), 'F_pvalue': np.float64(0.0014002645988316735)}

=== Step 2.6 Decision ===
- Multicollinearity: NO (VIF<=10) -> keep original X.
- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.
- Next step: Main Estimation (Two-way FE) using clustered SE.


In [72]:

# =========================
# STEP 3 — MAIN ESTIMATION
# Two-way FE + Clustered SE (province)
# =========================
Y = "agr_share"                      # bạn đổi thành "share_urban" nếu muốn
X = ["lhdi", "ldensity", "lepe", "lipi"]

df_est = df[["province", "year", Y] + X].dropna().copy()
df_est["year"] = pd.to_numeric(df_est["year"], errors="coerce").astype(int)

panel = df_est.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]

fe_2way = PanelOLS(
    y, X_df,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
).fit(
    cov_type="clustered",
    cluster_entity=True
)

print("=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===")
print(fe_2way.summary)


result_table = pd.DataFrame({
    "coef": fe_2way.params,
    "std_err": fe_2way.std_errors,
    "t": fe_2way.tstats,
    "p_value": fe_2way.pvalues
}).sort_values("p_value")

print("\n=== Coefficient table (sorted by p-value) ===")
print(result_table)


=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===
                          PanelOLS Estimation Summary                           
Dep. Variable:              agr_share   R-squared:                        0.0021
Estimator:                   PanelOLS   R-squared (Between):             -2.5053
No. Observations:                 259   R-squared (Within):              -0.0010
Date:                Mon, Dec 22 2025   R-squared (Overall):             -2.5810
Time:                        09:53:38   Log-likelihood                    605.28
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.1035
Entities:                          56   P-value                           0.9812
Avg Obs:                       4.6250   Distribution:                   F(4,195)
Min Obs:                       1.0000                                           
Max Obs:                   

**TEST FOR FORESTRY SHARE**

In [73]:
# =========================
# STEP 2 — DIAGNOSTICS (share_urban)
# =========================
Y = "forestry_share"
X = ["lhdi", "ldensity", "lepe", "lipi"]

# 2.0: prepare panel data
df_m = df[["province", "year", Y] + X].dropna().copy()
df_m["year"] = pd.to_numeric(df_m["year"], errors="coerce").astype(int)

panel = df_m.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]


# -------------------------
# Multicollinearity (VIF>10)
# -------------------------
def vif_report(X_df: pd.DataFrame):
    Xv = X_df.copy()
    Xv = Xv.dropna()
    Xv = sm.add_constant(Xv)
    out = []
    for i, col in enumerate(Xv.columns):
        out.append([col, variance_inflation_factor(Xv.values, i)])
    return pd.DataFrame(out, columns=["var","VIF"]).sort_values("VIF", ascending=False)

vif_df = vif_report(X_df)
print("\n=== Step 2.2 VIF ===")
print(vif_df)

VIF_THRESHOLD = 10
need_pca = vif_df.query("var != 'const'")["VIF"].max() > VIF_THRESHOLD

if need_pca:
    print("\n>>> VIF > 10 detected -> Step 2.2b PCA grouping")
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X_df.values)

    pca = PCA(n_components=min(2, X_df.shape[1]))
    pcs = pca.fit_transform(X_std)

    print("Explained variance ratio:", pca.explained_variance_ratio_)


    X_pca = pd.DataFrame(pcs, index=X_df.index, columns=["pc1","pc2"][:pcs.shape[1]])
    panel_pca = panel.copy()
    for c in X_pca.columns:
        panel_pca[c] = X_pca[c]
else:
    panel_pca = None


# -------------------------
# Serial correlation test ( simple AR(1) on residuals)
# -------------------------

resid_df = resid.to_frame("resid").reset_index()  # columns: province, year, resid
resid_df["resid_lag1"] = resid_df.groupby("province")["resid"].shift(1)
ar_df = resid_df.dropna(subset=["resid","resid_lag1"]).copy()

if len(ar_df) >= 10:
    X_ar = sm.add_constant(ar_df["resid_lag1"])
    ar_model = sm.OLS(ar_df["resid"], X_ar).fit()
    p_serial = ar_model.pvalues.get("resid_lag1", np.nan)
    print("\n=== Step 2.4 Serial correlation (AR(1) on residuals) ===")
    print(ar_model.summary().tables[1])
    print(f"Serial correlation p-value (lag residual) = {p_serial:.6f}")
else:
    p_serial = np.nan
    print("\n=== Step 2.4 Serial correlation ===")
    print("Not enough data to test serial correlation.")

# -------------------------
# Heteroskedasticity test (Breusch–Pagan on FE residuals vs regressors)
# -------------------------

if len(resid) == len(X_df):
    exog_bp = sm.add_constant(X_df)
    bp_stat, bp_p, f_stat, f_p = het_breuschpagan(resid.values, exog_bp.values)
    print("\n=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===")
    print({"LM_pvalue": bp_p, "F_pvalue": f_p})
else:
    print("\n=== Step 2.5 Heteroskedasticity ===")
    print("Residual/exog length mismatch (due to dropped absorbed vars). Skip BP or recompute on aligned index.")

# -------------------------
# Decision
# -------------------------
print("\n=== Step 2.6 Decision ===")
if need_pca:
    print("- Multicollinearity: YES (VIF>10) -> consider using PCA components (pc1/pc2) for estimation.")
else:
    print("- Multicollinearity: NO (VIF<=10) -> keep original X.")

if not np.isnan(p_serial) and p_serial < 0.05:
    print("- Serial correlation: YES (p<0.05) -> use Cluster SE by province (recommended).")
else:
    print("- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.")

print("- Next step: Main Estimation (Two-way FE) using clustered SE.")


=== Step 2.2 VIF ===
        var          VIF
0     const  7400.940745
1      lhdi     2.649627
2  ldensity     1.948292
3      lepe     1.874003
4      lipi     1.033488

=== Step 2.4 Serial correlation (AR(1) on residuals) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.681e-06   3.48e-05      0.278      0.781   -5.89e-05    7.83e-05
resid_lag1    -0.0051      0.040     -0.127      0.899      -0.084       0.074
Serial correlation p-value (lag residual) = 0.898763

=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===
{'LM_pvalue': np.float64(0.001625707667532991), 'F_pvalue': np.float64(0.0014002645988316735)}

=== Step 2.6 Decision ===
- Multicollinearity: NO (VIF<=10) -> keep original X.
- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.
- Next step: Main Estimation (Two-way FE) using clustered SE.


In [75]:

# =========================
# STEP 3 — MAIN ESTIMATION
# Two-way FE + Clustered SE (province)
# =========================
Y = "forestry_share"                      # bạn đổi thành "share_urban" nếu muốn
X = ["lhdi", "ldensity", "lepe", "lipi"]

df_est = df[["province", "year", Y] + X].dropna().copy()
df_est["year"] = pd.to_numeric(df_est["year"], errors="coerce").astype(int)

panel = df_est.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]

fe_2way = PanelOLS(
    y, X_df,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
).fit(
    cov_type="clustered",
    cluster_entity=True
)

print("=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===")
print(fe_2way.summary)


result_table = pd.DataFrame({
    "coef": fe_2way.params,
    "std_err": fe_2way.std_errors,
    "t": fe_2way.tstats,
    "p_value": fe_2way.pvalues
}).sort_values("p_value")

print("\n=== Coefficient table (sorted by p-value) ===")
print(result_table)


=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===
                          PanelOLS Estimation Summary                           
Dep. Variable:         forestry_share   R-squared:                        0.0185
Estimator:                   PanelOLS   R-squared (Between):             -2.1798
No. Observations:                 259   R-squared (Within):              -0.0093
Date:                Mon, Dec 22 2025   R-squared (Overall):             -2.1356
Time:                        10:00:06   Log-likelihood                    790.35
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.9179
Entities:                          56   P-value                           0.4545
Avg Obs:                       4.6250   Distribution:                   F(4,195)
Min Obs:                       1.0000                                           
Max Obs:                   

In [77]:
# =========================
# STEP 2 — DIAGNOSTICS (share_urban)
# =========================
Y = "special_land"
X = ["lhdi", "ldensity", "lepe", "lipi"]

# 2.0: prepare panel data
df_m = df[["province", "year", Y] + X].dropna().copy()
df_m["year"] = pd.to_numeric(df_m["year"], errors="coerce").astype(int)

panel = df_m.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]


# -------------------------
# Multicollinearity (VIF>10)
# -------------------------
def vif_report(X_df: pd.DataFrame):
    Xv = X_df.copy()
    Xv = Xv.dropna()
    Xv = sm.add_constant(Xv)
    out = []
    for i, col in enumerate(Xv.columns):
        out.append([col, variance_inflation_factor(Xv.values, i)])
    return pd.DataFrame(out, columns=["var","VIF"]).sort_values("VIF", ascending=False)

vif_df = vif_report(X_df)
print("\n=== Step 2.2 VIF ===")
print(vif_df)

VIF_THRESHOLD = 10
need_pca = vif_df.query("var != 'const'")["VIF"].max() > VIF_THRESHOLD

if need_pca:
    print("\n>>> VIF > 10 detected -> Step 2.2b PCA grouping")
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X_df.values)

    pca = PCA(n_components=min(2, X_df.shape[1]))
    pcs = pca.fit_transform(X_std)

    print("Explained variance ratio:", pca.explained_variance_ratio_)


    X_pca = pd.DataFrame(pcs, index=X_df.index, columns=["pc1","pc2"][:pcs.shape[1]])
    panel_pca = panel.copy()
    for c in X_pca.columns:
        panel_pca[c] = X_pca[c]
else:
    panel_pca = None


# -------------------------
# Serial correlation test ( simple AR(1) on residuals)
# -------------------------

resid_df = resid.to_frame("resid").reset_index()  # columns: province, year, resid
resid_df["resid_lag1"] = resid_df.groupby("province")["resid"].shift(1)
ar_df = resid_df.dropna(subset=["resid","resid_lag1"]).copy()

if len(ar_df) >= 10:
    X_ar = sm.add_constant(ar_df["resid_lag1"])
    ar_model = sm.OLS(ar_df["resid"], X_ar).fit()
    p_serial = ar_model.pvalues.get("resid_lag1", np.nan)
    print("\n=== Step 2.4 Serial correlation (AR(1) on residuals) ===")
    print(ar_model.summary().tables[1])
    print(f"Serial correlation p-value (lag residual) = {p_serial:.6f}")
else:
    p_serial = np.nan
    print("\n=== Step 2.4 Serial correlation ===")
    print("Not enough data to test serial correlation.")

# -------------------------
# Heteroskedasticity test (Breusch–Pagan on FE residuals vs regressors)
# -------------------------

if len(resid) == len(X_df):
    exog_bp = sm.add_constant(X_df)
    bp_stat, bp_p, f_stat, f_p = het_breuschpagan(resid.values, exog_bp.values)
    print("\n=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===")
    print({"LM_pvalue": bp_p, "F_pvalue": f_p})
else:
    print("\n=== Step 2.5 Heteroskedasticity ===")
    print("Residual/exog length mismatch (due to dropped absorbed vars). Skip BP or recompute on aligned index.")

# -------------------------
# Decision
# -------------------------
print("\n=== Step 2.6 Decision ===")
if need_pca:
    print("- Multicollinearity: YES (VIF>10) -> consider using PCA components (pc1/pc2) for estimation.")
else:
    print("- Multicollinearity: NO (VIF<=10) -> keep original X.")

if not np.isnan(p_serial) and p_serial < 0.05:
    print("- Serial correlation: YES (p<0.05) -> use Cluster SE by province (recommended).")
else:
    print("- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.")

print("- Next step: Main Estimation (Two-way FE) using clustered SE.")


=== Step 2.2 VIF ===
        var          VIF
0     const  7344.701382
1      lhdi     2.649575
2  ldensity     1.931260
3      lepe     1.816888
4      lipi     1.027967

=== Step 2.4 Serial correlation (AR(1) on residuals) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.681e-06   3.48e-05      0.278      0.781   -5.89e-05    7.83e-05
resid_lag1    -0.0051      0.040     -0.127      0.899      -0.084       0.074
Serial correlation p-value (lag residual) = 0.898763

=== Step 2.5 Heteroskedasticity ===
Residual/exog length mismatch (due to dropped absorbed vars). Skip BP or recompute on aligned index.

=== Step 2.6 Decision ===
- Multicollinearity: NO (VIF<=10) -> keep original X.
- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.
- Next step: Main Estimation (Two-way FE) using clustered SE.


**TEST FOR SPECIAL LAND SHARE**

In [78]:
# =========================
# STEP 2 — DIAGNOSTICS (share_urban)
# =========================
Y = "special_share"
X = ["lhdi", "ldensity", "lepe", "lipi"]

# 2.0: prepare panel data
df_m = df[["province", "year", Y] + X].dropna().copy()
df_m["year"] = pd.to_numeric(df_m["year"], errors="coerce").astype(int)

panel = df_m.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]


# -------------------------
# Multicollinearity (VIF>10)
# -------------------------
def vif_report(X_df: pd.DataFrame):
    Xv = X_df.copy()
    Xv = Xv.dropna()
    Xv = sm.add_constant(Xv)
    out = []
    for i, col in enumerate(Xv.columns):
        out.append([col, variance_inflation_factor(Xv.values, i)])
    return pd.DataFrame(out, columns=["var","VIF"]).sort_values("VIF", ascending=False)

vif_df = vif_report(X_df)
print("\n=== Step 2.2 VIF ===")
print(vif_df)

VIF_THRESHOLD = 10
need_pca = vif_df.query("var != 'const'")["VIF"].max() > VIF_THRESHOLD

if need_pca:
    print("\n>>> VIF > 10 detected -> Step 2.2b PCA grouping")
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X_df.values)

    pca = PCA(n_components=min(2, X_df.shape[1]))
    pcs = pca.fit_transform(X_std)

    print("Explained variance ratio:", pca.explained_variance_ratio_)


    X_pca = pd.DataFrame(pcs, index=X_df.index, columns=["pc1","pc2"][:pcs.shape[1]])
    panel_pca = panel.copy()
    for c in X_pca.columns:
        panel_pca[c] = X_pca[c]
else:
    panel_pca = None


# -------------------------
# Serial correlation test ( simple AR(1) on residuals)
# -------------------------

resid_df = resid.to_frame("resid").reset_index()  # columns: province, year, resid
resid_df["resid_lag1"] = resid_df.groupby("province")["resid"].shift(1)
ar_df = resid_df.dropna(subset=["resid","resid_lag1"]).copy()

if len(ar_df) >= 10:
    X_ar = sm.add_constant(ar_df["resid_lag1"])
    ar_model = sm.OLS(ar_df["resid"], X_ar).fit()
    p_serial = ar_model.pvalues.get("resid_lag1", np.nan)
    print("\n=== Step 2.4 Serial correlation (AR(1) on residuals) ===")
    print(ar_model.summary().tables[1])
    print(f"Serial correlation p-value (lag residual) = {p_serial:.6f}")
else:
    p_serial = np.nan
    print("\n=== Step 2.4 Serial correlation ===")
    print("Not enough data to test serial correlation.")

# -------------------------
# Heteroskedasticity test (Breusch–Pagan on FE residuals vs regressors)
# -------------------------

if len(resid) == len(X_df):
    exog_bp = sm.add_constant(X_df)
    bp_stat, bp_p, f_stat, f_p = het_breuschpagan(resid.values, exog_bp.values)
    print("\n=== Step 2.5 Heteroskedasticity (Breusch–Pagan) ===")
    print({"LM_pvalue": bp_p, "F_pvalue": f_p})
else:
    print("\n=== Step 2.5 Heteroskedasticity ===")
    print("Residual/exog length mismatch (due to dropped absorbed vars). Skip BP or recompute on aligned index.")

# -------------------------
# Decision
# -------------------------
print("\n=== Step 2.6 Decision ===")
if need_pca:
    print("- Multicollinearity: YES (VIF>10) -> consider using PCA components (pc1/pc2) for estimation.")
else:
    print("- Multicollinearity: NO (VIF<=10) -> keep original X.")

if not np.isnan(p_serial) and p_serial < 0.05:
    print("- Serial correlation: YES (p<0.05) -> use Cluster SE by province (recommended).")
else:
    print("- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.")

print("- Next step: Main Estimation (Two-way FE) using clustered SE.")


=== Step 2.2 VIF ===
        var          VIF
0     const  7344.701382
1      lhdi     2.649575
2  ldensity     1.931260
3      lepe     1.816888
4      lipi     1.027967

=== Step 2.4 Serial correlation (AR(1) on residuals) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.681e-06   3.48e-05      0.278      0.781   -5.89e-05    7.83e-05
resid_lag1    -0.0051      0.040     -0.127      0.899      -0.084       0.074
Serial correlation p-value (lag residual) = 0.898763

=== Step 2.5 Heteroskedasticity ===
Residual/exog length mismatch (due to dropped absorbed vars). Skip BP or recompute on aligned index.

=== Step 2.6 Decision ===
- Multicollinearity: NO (VIF<=10) -> keep original X.
- Serial correlation: NO (or not tested) -> Cluster SE still recommended in panel.
- Next step: Main Estimation (Two-way FE) using clustered SE.


In [79]:

# =========================
# STEP 3 — MAIN ESTIMATION
# Two-way FE + Clustered SE (province)
# =========================
Y = "special_share"                      # bạn đổi thành "share_urban" nếu muốn
X = ["lhdi", "ldensity", "lepe", "lipi"]

df_est = df[["province", "year", Y] + X].dropna().copy()
df_est["year"] = pd.to_numeric(df_est["year"], errors="coerce").astype(int)

panel = df_est.set_index(["province", "year"]).sort_index()
y = panel[Y]
X_df = panel[X]

fe_2way = PanelOLS(
    y, X_df,
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
).fit(
    cov_type="clustered",
    cluster_entity=True
)

print("=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===")
print(fe_2way.summary)


result_table = pd.DataFrame({
    "coef": fe_2way.params,
    "std_err": fe_2way.std_errors,
    "t": fe_2way.tstats,
    "p_value": fe_2way.pvalues
}).sort_values("p_value")

print("\n=== Coefficient table (sorted by p-value) ===")
print(result_table)


=== Step 3: Two-way Fixed Effects (Province + Year), Clustered SE by Province ===
                          PanelOLS Estimation Summary                           
Dep. Variable:          special_share   R-squared:                        0.0686
Estimator:                   PanelOLS   R-squared (Between):             -496.92
No. Observations:                 267   R-squared (Within):               0.0118
Date:                Mon, Dec 22 2025   R-squared (Overall):             -482.94
Time:                        10:02:54   Log-likelihood                    733.59
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.7385
Entities:                          56   P-value                           0.0059
Avg Obs:                       4.7679   Distribution:                   F(4,203)
Min Obs:                       1.0000                                           
Max Obs:                   

**RESAMPLING**

In [80]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS

# =========================
# JACKKNIFE: Leave-one-province-out
# Two-way FE (province + year)
# Outcomes: 4 shares
# Regressors: 4 logged covariates
# =========================

outcomes = ["special_share", "resi_share", "agr_share", "forestry_share"]
regressors = ["lhdi", "ldensity", "lepe", "lipi"]

# 0) basic clean
base_cols = ["province", "year"] + outcomes + regressors
d0 = df[base_cols].dropna().copy()
d0["year"] = pd.to_numeric(d0["year"], errors="coerce").astype(int)

# 1) helper: fit two-way FE and return params
def twfe_params(data_long: pd.DataFrame, y_name: str, x_names: list[str]) -> pd.Series:
    panel_dat = data_long.set_index(["province", "year"]).sort_index()
    y_vec = panel_dat[y_name]
    x_mat = panel_dat[x_names]
    fit = PanelOLS(
        y_vec, x_mat,
        entity_effects=True,
        time_effects=True,
        drop_absorbed=True
    ).fit(cov_type="unadjusted")  # jackknife tự tạo phân phối -> để unadjusted
    return fit.params

# 2) full-sample estimates for each Y
full_est = {}
for yvar in outcomes:
    full_est[yvar] = twfe_params(d0, yvar, regressors)

# 3) leave-one-province-out loop
units = d0["province"].unique()
rows = []

for yvar in outcomes:
    for u in units:
        d_minus = d0[d0["province"] != u]
        try:
            b = twfe_params(d_minus, yvar, regressors)
            rec = {"outcome": yvar, "left_out_province": u}
            # đảm bảo đủ 4 hệ số; nếu biến bị absorbed -> NaN
            for x in regressors:
                rec[x] = float(b[x]) if x in b.index else np.nan
            rows.append(rec)
        except Exception:
            # nếu mô hình fail khi bỏ tỉnh này, vẫn ghi nhận
            rows.append({"outcome": yvar, "left_out_province": u, **{x: np.nan for x in regressors}})

jack_df = pd.DataFrame(rows)

# 4) summarize stability
summary_rows = []
for yvar in outcomes:
    sub = jack_df[jack_df["outcome"] == yvar].copy()

    for x in regressors:
        vals = sub[x].dropna()
        full_beta = float(full_est[yvar][x]) if x in full_est[yvar].index else np.nan

        if len(vals) == 0:
            summary_rows.append([yvar, x, full_beta, np.nan, np.nan, np.nan, np.nan, 0, np.nan])
            continue

        # sign stability: % cùng dấu với full estimate
        if np.isnan(full_beta) or full_beta == 0:
            sign_match = np.nan
        else:
            sign_match = float((np.sign(vals) == np.sign(full_beta)).mean())

        summary_rows.append([
            yvar,
            x,
            full_beta,
            float(vals.mean()),
            float(vals.std(ddof=1)) if len(vals) > 1 else 0.0,
            float(vals.quantile(0.05)),
            float(vals.quantile(0.95)),
            int(vals.shape[0]),
            sign_match
        ])

stability = pd.DataFrame(
    summary_rows,
    columns=["outcome","regressor","full_beta","jack_mean","jack_sd","p05","p95","n_runs","sign_match_rate"]
)

print("=== Jackknife stability summary (Two-way FE) ===")
print(stability.sort_values(["outcome","regressor"]))


=== Jackknife stability summary (Two-way FE) ===
           outcome regressor  full_beta  jack_mean   jack_sd       p05  \
9        agr_share  ldensity  -0.082119  -0.081879  0.019595 -0.106676   
10       agr_share      lepe   0.001948   0.001951  0.000623  0.001295   
8        agr_share      lhdi  -0.001016  -0.001239  0.019016 -0.016035   
11       agr_share      lipi   0.013532   0.013500  0.004023  0.009500   
13  forestry_share  ldensity  -0.086668  -0.086682  0.018884 -0.102908   
14  forestry_share      lepe  -0.004124  -0.004127  0.000410 -0.004605   
12  forestry_share      lhdi   0.076246   0.076213  0.007086  0.062240   
15  forestry_share      lipi   0.011563   0.011564  0.004293  0.008983   
5       resi_share  ldensity  -0.001466  -0.001476  0.001952 -0.002969   
6       resi_share      lepe   0.000065   0.000065  0.000024  0.000031   
4       resi_share      lhdi  -0.008246  -0.008265  0.001005 -0.009571   
7       resi_share      lipi  -0.002237  -0.002237  0.000131 -0